# Notebook for the Coursera Capstone Project 'Battle of the Neighboorhods'

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes

import pandas as pd
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.0               |             py_0         606 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

#### Getting data from FourSquare

In [2]:
# The code was removed by Watson Studio for sharing.

We build a GET request to FourSquare API. Here, we find coffee shop near Milan center, in a radius of 1000 meters. Here we build a request parameter and add it to url.

In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": VERSION,
    "section": "coffee",
    "near": "Milan",
    "radius": 1000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

### Data Analysis

In [4]:

d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

Now display data contained by their key, here we count how many coffe shop are in the bound of Milan center

In [6]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 45.4730833825003, 'lng': 9.200226988220452},
  'sw': {'lat': 45.454973385685356, 'lng': 9.17599138686897}},
 137)

Because we obtain nested object from API Call Let's print first item to get better idea of the data structure.

In [8]:
d["groups"][0]["items"][0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '58b5d3364e31c52f888a0bed',
  'name': 'Starbucks Reserve Roastery',
  'location': {'address': 'Piazza Cordusio 3',
   'crossStreet': 'Via Orefici',
   'lat': 45.46492026479228,
   'lng': 9.186152815818787,
   'labeledLatLngs': [{'label': 'display',
     'lat': 45.46492026479228,
     'lng': 9.186152815818787}],
   'postalCode': '20123',
   'cc': 'IT',
   'city': 'Milano',
   'state': 'Lombardia',
   'country': 'Italia',
   'formattedAddress': ['Piazza Cordusio 3 (Via Orefici)',
    '20123 Milano Lombardia',
    'Italia']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': [

In [9]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]["venue"].keys()

number of items: 100


dict_keys(['id', 'name', 'location', 'categories', 'photos'])

From retrieved venues we build a new pandas dataframe

In [10]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["formattedAddress"]
    if not "postalCode" in location: #if we have no postal code skip this shop (maybe there is a wrong address)
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address","postalcode", "lat", "lng"])
print("found %i coffee" % len(df))
df.head()

found 75 coffee


,uid,name,shortname,address,postalcode,lat,lng
0,58b5d3364e31c52f888a0bed,Starbucks Reserve Roastery,Coffee Shop,"[Piazza Cordusio 3 (Via Orefici), 20123 Milano...",20123,45.464920,9.186153
1,5485c0d7498e225e0587b79b,Panini Durini,Sandwiches,"[Via Mengoni 4, 20121 Milano Lombardia, Italia]",20121,45.465238,9.188590
2,59c2232bd48ec17dd4178369,Lavazza Coffee Design,Coffee Shop,"[Piazza San Fedele, 2, 20122 Milano Lombardia,...",20122,45.466274,9.190975
3,576828cc498ec1d7fb395d70,Caffè Napoli,Café,"[Via Gaetano Giardino 1, 20123 Milano Lombardi...",20123,45.462816,9.189023
4,4be181c240d676b02de504ee,Princi,Bakery,"[Via Speronari, 6, 20123 Milano Lombardia, Ita...",20123,45.463029,9.187968


In [ ]:
## Kernel Densityu 

Some density based on estimator is giving a good tip where to start a new coffee shop.  There is a HeatMap to visualize all the existing coffee shops to same map.

In [11]:
milan_center = d["geocode"]["center"]
milan_center

{'lat': 45.46427, 'lng': 9.18951}

In [12]:
from folium import plugins

# create map of milan using latitude and longitude values
map_milan = folium.Map(location=[milan_center["lat"], milan_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_milan)

add_markers(df)
hm_data = df[["lat", "lng"]].values.tolist()
map_milan.add_child(plugins.HeatMap(hm_data))

map_milan

## Results

Based on these results, one possibly good location for the new Coffee Shop would be in piazza fontana, at the end of a long way that connect Milan universtity and the behind of Duomo di Milano Cathedral 

In [13]:
lat = 45.46377
lng = 9.19356
map_milan = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our new Coffee Shop!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_milan)
map_milan